In [1]:
import time
from model import *
from model_logging import *
from model_trainer import *

In [2]:
dtype = torch.FloatTensor
ltype = torch.LongTensor
dataset = '../final_dataset_original/'
use_cuda=True
if use_cuda:
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

In [3]:
model = ParallelModel()
model.dtype = dtype
if use_cuda:
    model.cuda()
else:
    model.cpu()

In [4]:
a = Variable(torch.FloatTensor(16,1,96,1366)).cuda()

In [5]:
model(a)

torch.Size([16, 128])
torch.Size([16, 128])


Variable containing:

Columns 0 to 9 
 0.4534  0.4754  0.5290  0.5302  0.4786  0.5397  0.5153  0.5090  0.5812  0.4574
 0.4817  0.4651  0.5056  0.5974  0.5184  0.5102  0.4810  0.5164  0.5426  0.4149
 0.4710  0.4268  0.4772  0.4916  0.4541  0.5571  0.5090  0.5287  0.5630  0.4602
 0.4529  0.5008  0.4739  0.5291  0.4974  0.5021  0.5115  0.5259  0.5436  0.4986
 0.4579  0.4691  0.5115  0.5495  0.5201  0.5092  0.5155  0.5216  0.5137  0.4555
 0.4983  0.5178  0.4838  0.5473  0.4711  0.5139  0.4504  0.4728  0.5161  0.4652
 0.4402  0.4622  0.5092  0.4878  0.5068  0.5348  0.4536  0.5327  0.5118  0.4297
 0.4616  0.4310  0.4945  0.4940  0.5335  0.5014  0.4996  0.4628  0.5534  0.4655
 0.4715  0.5165  0.5058  0.5352  0.4960  0.5499  0.5440  0.5189  0.4860  0.4911
 0.4752  0.4534  0.4937  0.4340  0.4479  0.5237  0.4928  0.5311  0.5202  0.4821
 0.5006  0.4264  0.4672  0.4955  0.4354  0.4618  0.5015  0.4829  0.5453  0.4985
 0.5254  0.4668  0.4549  0.4858  0.4853  0.5351  0.5307  0.5146  0.5383  0.4777
 0

In [4]:

X_train = np.load(dataset+'train_x.npy')
Y_train = np.load(dataset+'train_y.npy')

Y_train = np.array(Y_train,dtype=int)

In [5]:
X_train = torch.FloatTensor(X_train)
Y_train = torch.LongTensor(Y_train)
train_data = torch.utils.data.TensorDataset(data_tensor=X_train,target_tensor=Y_train)

In [6]:
X_val = np.load(dataset+'valid_x.npy')
Y_val = np.load(dataset+'valid_y.npy')

Y_val = np.array(Y_val,dtype=int)

X_val = torch.FloatTensor(X_val)
Y_val = torch.LongTensor(Y_val)
valid_data = torch.utils.data.TensorDataset(data_tensor=X_val,target_tensor=Y_val)

In [34]:
trainer = ModelTrainer(model=model.cuda(),
                      lr = 0.0001,
                      dataset = train_data,
                      validset = valid_data,
                      weight_decay = 0.0,
                      snapshot_path = 'snapshots',
                      snapshot_name = 'saber_model_mix',
                      snapshot_interval = 150,
                      snapshot_thresh = 9000,
                      dtype = dtype,
                      ltype = ltype)

In [35]:
tic = time.time()
trainer.train(batch_size=16,
              epochs=1,
              continue_training_at_step = 14700)
toc = time.time()
print 'Training took %d seconds' %(toc-tic)

epoch  0
loss at step 14750: 0.155962719023
loss at step 14800: 0.108455591649
loss at step 14850: 0.108795725554
loss at step 14900: 0.105810401738
loss at step 14950: 0.112448172122
loss at step 15000: 0.111846125573
validation loss: 0.12774467242
loss at step 15050: 0.108999637663
loss at step 15100: 0.111505176723
loss at step 15150: 0.112539089769
loss at step 15200: 0.109097419232
loss at step 15250: 0.109593453258
loss at step 15300: 0.108176848441
loss at step 15350: 0.105248434991
loss at step 15400: 0.108290463984
loss at step 15450: 0.110931118727
loss at step 15500: 0.115726265162
loss at step 15550: 0.110347103328
loss at step 15600: 0.111922775656
loss at step 15650: 0.109529827386
loss at step 15700: 0.110863737017
loss at step 15750: 0.110092524737
loss at step 15800: 0.109335795939
loss at step 15850: 0.109442203939
loss at step 15900: 0.111785297245
Training took 378 seconds


In [78]:
model = torch.load('snapshots/saber_model_mix_15000')

In [9]:
from sklearn.metrics import roc_auc_score

In [10]:
x_test = np.load(dataset+'test_x.npy')
y_test = np.load(dataset+'test_y.npy')
y_test = np.array(y_test,dtype=int)

In [11]:
x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)
test_data = torch.utils.data.TensorDataset(data_tensor=x_test,target_tensor=y_test)
test_data = torch.utils.data.DataLoader(test_data,
                                         batch_size = 8,
                                         shuffle = False,
                                         num_workers = 8,
                                         pin_memory=False)

In [79]:
y_pred = None
for x,y in iter(test_data):
    x = Variable(x).cuda()
    outputs = model(x).view(-1,50)
    if y_pred is None:
        y_pred = outputs.cpu().data
    else:
        y_pred = torch.cat((y_pred,outputs.cpu().data),dim=0)

y_pred_np = y_pred.numpy()
y_test_np = y_test.numpy()


test_AUC = roc_auc_score(y_test_np,y_pred_np)

test_AUC

0.88411909655492382

In [45]:
y_pred_np[0]

array([  2.68557332e-02,   2.40016179e-05,   9.86941159e-04,
         5.18169761e-01,   2.86673661e-04,   6.48300024e-03,
         3.15401703e-04,   5.05167292e-03,   2.33487622e-03,
         5.50687253e-01,   1.21358910e-03,   1.08063174e-03,
         6.54161302e-03,   5.04257157e-04,   5.39053883e-03,
         4.99732676e-04,   1.14320721e-02,   3.56493652e-01,
         1.06931627e-02,   4.54413772e-01,   2.57599889e-03,
         7.55597603e-06,   8.49293843e-02,   1.13787763e-02,
         1.12587493e-03,   3.03875878e-02,   5.02398296e-04,
         1.14195189e-03,   8.69585387e-03,   1.45004932e-02,
         4.02112395e-01,   6.13199736e-05,   3.89797240e-02,
         3.54442117e-03,   9.43347459e-06,   3.50518472e-04,
         2.28874665e-02,   1.81105211e-01,   8.87709931e-02,
         6.42595291e-02,   2.54248572e-03,   9.47654068e-01,
         3.74063035e-03,   5.32258164e-05,   1.57724917e-02,
         7.96913719e-06,   5.10627478e-02,   1.90562394e-03,
         7.47813829e-05,

In [1]:
torch.save(model,'snapshots/saber_model_25000')

NameError: name 'torch' is not defined

In [23]:
x,y = iter(valid_data).next()
x = Variable(x.contiguous()).cuda().view(1,1366,96)
y = Variable(y.type(torch.FloatTensor)).cuda()

In [17]:
test_AUC


0.87868866869580142

In [25]:
F.binary_cross_entropy(out,y)

Variable containing:
 1.3899
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1. wavenet: dilation_depth=6, n_blocks=5, output = 64
   CNN: feature maps = all 128 , output = 64
   Linear: 64+64 -> 512 -> 50
   AUC: 88.13
2. wavenet: dilation_depth=8, n_blocks=4, output = 64
   CNN: feature maps = all 128 , output = 64
   Linear: 64+64 -> 512 -> 50
   AUC: 88.07
3. wavenet: dilation_depth=6, n_blocks=5, output = 128
   CNN: feature maps = all 128 , output = 128
   Linear: 128+128 -> 512 -> 50
   AUC: 88.57

4. without dropout with SEnet: 88.45
   converge faster，about 0.31seconds per step

5. without dropout and SEnet: 88.52
   about 0.36 seconds per step
   
6. replace the activation function with relu at last two linear layer: 88.71 